Title of Project and Explanation

## Libraries <br>

In [566]:
# Pandas is required as I will be organising the data extracted from the various sources into Dataframes
import pandas as pd

# Requests is used to interact with APIs
import requests

# Numpy is used for some data filtering
import numpy as np

In [567]:
# This string is used as an argument to the IMF-related functions and serves as a reminder which period I am looking at
data_periods = '2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022'

## Functions<br>
These functions will be used throughout the script to get data through APIs and the clean it into a format from which it can be merged to the main dataset

In [568]:
def create_irena_period_data(df: pd.DataFrame, period: str, year: int, remove_country: bool =False) -> pd.DataFrame:
    """
    This function creates the IRENA dataset. This is the dataset which will contain:
    a) The target variable (change in % of renewable sources in total electricity generation)
    b) The amount of renewable electricity generation at the start of the period in question
    This function should be run twice, for both the previous and current periods, so a change in renewable % can be calculated
    :param df: The IRENA dataset in dataframe format
    :param period: 'CUR' or 'PRE' as required
    :param year: The year of observations to be extracted from the dataframe
    :param remove_country: only include the country column on one of the datasets (either 'CUR' or 'PRE') to prevent duplication of columns
    :return: Dataframe with both previous and current electricity generation data
    """
    df = df[irena_data['Year'] == year].groupby(by=['Country', 'ISO3 code', 'RE or Non-RE'])['Electricity Generation (GWh)'].sum().reset_index(name='Electricity Generation (GWh)')
    
    # Create columns for both Renewable & Non-Renewable as right now they are by row
    
    # Create the Renewable DataFrame
    renewable = df[df['RE or Non-RE'] == 'Total Renewable']
    renewable = renewable.groupby(by=['Country', 'ISO3 code'])['Electricity Generation (GWh)'].sum().reset_index()
    renewable = renewable.rename(columns={'Electricity Generation (GWh)': 'renewable'})
    
    # Create the Non-Renewable DataFrame
    non_renewable = df[df['RE or Non-RE'] == 'Total Non-Renewable']
    non_renewable = non_renewable.groupby(by=['Country', 'ISO3 code'])['Electricity Generation (GWh)'].sum().reset_index()
    non_renewable = non_renewable.rename(columns={'Electricity Generation (GWh)': 'non-renewable'})
    
    # Join them together
    grouped_irena = pd.merge(left=renewable,
                        right = non_renewable,
                        left_on=['Country', 'ISO3 code'],
                        right_on=['Country', 'ISO3 code'],
                        how='outer')
    
    # Two new fields are added which detail the total electricity generation and the percentage of this which is from renewable sources
    df = grouped_irena
    df['total_generation'] = df['renewable'] + df['non-renewable']
    df['renewable_mix'] = np.where(df['renewable'].isna(), 0,
                                   np.where(df['non-renewable'].isna(), 100, 
                                            (df['renewable'] / df['total_generation']) * 100))
    
    # Finally, columns are removed and/or renamed as required
    if remove_country:
        df = df[['Country', 'ISO3 code', 'renewable', 'renewable_mix']]
        df = df.rename(columns={'Country': 'country', 
                                'ISO3 code': 'iso3_code', 
                                'renewable': f'{period}_renewable_generation', 
                                'renewable_mix': f'{period}_renewable_mix'})
    else:
        df = df[['ISO3 code', 'renewable', 'renewable_mix']]
        df = df.rename(columns={'ISO3 code': 'iso3_code', 
                                'renewable': f'{period}_renewable_generation', 
                                'renewable_mix': f'{period}_renewable_mix'})
    
    return df

In [569]:
def get_world_bank_data(indicator: str, file_format: str, date: str) -> list:
    """
    This function connects to the World Bank API and extracts data based on arguments provided
    :param indicator: Each dataset has a corresponding indicator which is used here to direct the request to the correct endpoint
    :param file_format: Allows the user to select the format upon which the data should be returned (usually json)
    :param date: The year in question
    :return: Json file containing the required data based on the arguments
    """
    base_url = f"http://api.worldbank.org/v2/country/all/indicator/{indicator}?format={file_format}&date={date}"
    all_data = []
    page = 1
    
    # As the World Bank datasets are paginated, the function must cycle through the pages and append the data into one object
    while True:
        url = f'{base_url}&page={page}'
        response = requests.get(url)
        if response.status_code == 200:
            json_data = response.json()
            if len(json_data) >1 and json_data[1]:
                all_data.extend(json_data[1])
                page += 1
            else:
                break
        else:
            print(f'Error: {response.status_code}')
            break
            
    return all_data

In [570]:
def clean_world_bank_data(df: pd.DataFrame, value_name: str) -> pd.DataFrame:
    """
    This functions takes World Bank dataset and cleans/transforms it, ready to be merged to the main dataset
    :param df: World Bank dataset as required
    :param value_name: the default column name is 'value'. This needs to be updated to something more descriptive
    :return: Cleaned dataset
    """
    # Remove unneeded columns
    df = df[['countryiso3code', 'value']]
    
    # Format cells as required
    df['countryiso3code'] = df['countryiso3code'].astype(str)
    
    # Remove Nans and null values from the dataset
    df = df.dropna(subset = ['value'])
    
    # Rename columns
    df = df.rename(columns={'countryiso3code': 'iso3_code', 'value': value_name})
    
    return df

In [571]:
def get_imf_data(indicator: str, date: str) -> dict:
    """
    This function connects to the World Bank API and extracts data based on arguments provided
    :param indicator: Each dataset has a corresponding indicator which is used here to direct the request to the correct endpoint
    :param date: The year in question
    :return: Json file containing the required data based on the arguments
    """
    url = f"https://www.imf.org/external/datamapper/api/v1/{indicator}?periods={date}"
    response = requests.get(url)
    if response.status_code == 200:
        json_data = response.json()
        return json_data['values'][indicator]
    else:
        raise ValueError(f"error getting response from {url}: {response.status_code}")

In [572]:
def clean_imf_data(retrieved_data: dict) -> pd.DataFrame:
    """
    This functions takes IMF dataset and cleans/transforms it, ready to be merged to the main dataset.
    :param retrieved_data: IMF dataset as required.
    :return: Cleaned IMF dataset, ready to merged with the main dataset.
    """
    country_rows = []
    for country, years in retrieved_data.items():
        country_data = {'iso3_code': country}
        country_data.update(years)
        country_rows.append(country_data)
        
    return pd.DataFrame(country_rows)

## Step 1: Retrieve, Clean and Transform the IRENA dataset

In [573]:
irena_data = pd.read_excel(
    io=r'C:\Users\User\OneDrive\Documents\prof_practice\irena_energy_stats_2024.xlsx',
    sheet_name='Country'
)

In [574]:
# Create a dataframe for both comparison periods are required
pre_irena = create_irena_period_data(irena_data, 'pre',2013, False)
cur_irena = create_irena_period_data(irena_data, 'cur', 2022, True)

# Combine the periods into a single dataset
final_irena = pd.merge(left=pre_irena,
                       right=cur_irena,
                       left_on=['iso3_code'],
                       right_on=['iso3_code'],
                       how='inner')

# Create the target variable
final_irena['renewable_mix_change'] = final_irena['cur_renewable_mix'] - final_irena['pre_renewable_mix']

# Rename the columns
final_irena = final_irena[['country', 'iso3_code', 'pre_renewable_generation', 'renewable_mix_change']]

## Step 2: Add Median Age data from the World Health Organisation

In [575]:
# Import data from an XLSX file
who_data = pd.read_excel(io=r'C:\Users\User\OneDrive\Documents\prof_practice\who_median_age.xlsx', sheet_name='Data')

# Filter for the year required
who_data = who_data[who_data['Year'] == 2022]

# Remove unneeded columns
who_data = who_data[['Country ISO 3 code', 'Value Numeric']]

# Rename columns as required
who_data = who_data.rename(columns={'Country ISO 3 code': 'iso3_code', 'Value Numeric': 'median_age'})

In [576]:
# Combine the IRENA and WHO Datasets
complete_dataset = pd.merge(left=final_irena,
                       right=who_data,
                       left_on=['iso3_code'],
                       right_on=['iso3_code'],
                       how='inner')

## Step 3: Add GDP per Capita and Compulsory Education data from the World Bank

In [577]:
# Use the World Bank API to return the GDP Per Capita dataset
gdp_per_capita = pd.DataFrame.from_records(get_world_bank_data('NY.GDP.PCAP.CD', 'json', '2022'))

# Clean and format the data
gdp_per_capita = clean_world_bank_data(gdp_per_capita, 'gdp_per_capita')

# Add the dataset to the main DataFrame
complete_dataset = pd.merge(left=complete_dataset,
                       right=gdp_per_capita,
                       left_on=['iso3_code'],
                       right_on=['iso3_code'],
                       how='inner')

C:\Users\User\AppData\Local\Temp\ipykernel_11644\1452728300.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['countryiso3code'] = df['countryiso3code'].astype(str)


In [578]:
# Use the World Bank API to return the Compulsory Education dataset
comp_edu = pd.DataFrame.from_records(get_world_bank_data('SE.COM.DURS', 'json', '2022'))

# Clean and format the data
comp_edu = clean_world_bank_data(comp_edu, 'compulsory_education')

# Add the dataset to the main DataFrame
complete_dataset = pd.merge(left=complete_dataset,
                       right=comp_edu,
                       left_on=['iso3_code'],
                       right_on=['iso3_code'],
                       how='inner')

C:\Users\User\AppData\Local\Temp\ipykernel_11644\1452728300.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['countryiso3code'] = df['countryiso3code'].astype(str)


## Step 4: Add Average Inflation, average Central Gov Debt and AI Preparedness data from the IMF

In [579]:
# Retrieve and clean the data
inflation_json = get_imf_data('PCPIPCH', data_periods)
inflation_data = clean_imf_data(inflation_json)

# Calculate the metric average over the period in question
inflation_data['average_inflation'] = inflation_data.iloc[:, 1:].mean(axis=1)
final_inflation = inflation_data[['iso3_code', 'average_inflation']]

# Add the dataset to the main DataFrame
complete_dataset = pd.merge(left=complete_dataset,
                       right=final_inflation,
                       left_on=['iso3_code'],
                       right_on=['iso3_code'],
                       how='inner')

In [580]:
# Retrieve and clean the data
cg_debt_json = get_imf_data('CG_DEBT_GDP', data_periods)
cg_debt_data = clean_imf_data(cg_debt_json)

# Calculate the metric average over the period in question
cg_debt_data['average_net_debt'] = cg_debt_data.iloc[:, 1:].mean(axis=1)
final_cg_debt = cg_debt_data[['iso3_code', 'average_net_debt']]

# Add the dataset to the main DataFrame
complete_dataset = pd.merge(left=complete_dataset,
                       right=final_cg_debt,
                       left_on=['iso3_code'],
                       right_on=['iso3_code'],
                       how='inner')

In [581]:
# Retrieve and clean the data
ai_json = get_imf_data('AI_PI', '2023')
ai_data = clean_imf_data(ai_json)

final_ai = ai_data.rename(columns={'2023': 'ai_preparedness'})

# Add the dataset to the main DataFrame
complete_dataset = pd.merge(left=complete_dataset,
                       right=final_ai,
                       left_on=['iso3_code'],
                       right_on=['iso3_code'],
                       how='inner')

## Step 5: Prepare the Final, Completed Dataset

In [582]:
complete_dataset = complete_dataset[['country', 'gdp_per_capita', 'average_net_debt', 'average_inflation', 'median_age', 'compulsory_education', 'ai_preparedness', 'pre_renewable_generation', 'renewable_mix_change']]

In [583]:
complete_dataset

,country,gdp_per_capita,average_net_debt,average_inflation,median_age,compulsory_education,ai_preparedness,pre_renewable_generation,renewable_mix_change
0,Albania,6846.426143,69.731980,2.240000,35.943,9.0,0.526793,6960.030,0.000000
1,Algeria,4961.552577,28.773215,4.820000,27.983,10.0,0.370439,291.017,0.462285
2,Angola,2929.694455,67.575514,19.200000,16.390,10.0,0.259659,3638.585,30.309684
3,Argentina,13935.681111,69.525114,40.985714,31.739,14.0,0.473809,35092.719,4.005632
4,Armenia,6571.974455,53.822150,3.300000,35.815,12.0,0.492887,2177.200,8.447566
...,...,...,...,...,...,...,...,...,...
138,Uruguay,20691.973899,46.360337,8.410000,35.496,14.0,0.548641,9029.879,7.170168
139,Viet Nam,4116.385761,38.765303,3.200000,31.938,10.0,0.481869,58466.294,4.961942
140,Yemen,615.702079,72.253465,22.490000,18.118,9.0,0.253328,4.589,17.205222
141,Zambia,1447.123101,76.320599,11.480000,17.288,7.0,0.370720,13348.817,-10.947247
